In [ ]:
from pysekiro.collect_data import Data_collection

target = 'Genichiro_Ashina' # 苇名弦一郎
# target = 'Inner_Genichiro' # 心中的弦一郎
# target = 'Isshin,_the_Sword_Saint' # 剑圣一心
# target = 'Inner_Isshin' # 心中的一心

In [ ]:
c = Data_collection(target)
c.collect_data()

# 数据预览

可以先看这个演示 https://github.com/ricagj/pysekiro/blob/main/imgs/data_preview.gif

**注：以下展示的是我本地的数据集，我没有上传，所以要运行以下代码，请先自己收集至少一个数据集**

- The_battle_memory
    - Genichiro_Ashina （苇名弦一郎）
        - training_data-1.npy （第一个战斗数据）

In [ ]:
import os

import cv2
import numpy as np
import pandas as pd

target = 'Genichiro_Ashina' # 苇名弦一郎
# target = 'Inner_Genichiro' # 心中的弦一郎
# target = 'Isshin,_the_Sword_Saint' # 剑圣一心
# target = 'Inner_Isshin' # 心中的一心

path = os.path.join('The_battle_memory', target, f'training_data-{19}.npy')
if os.path.exists(path):
    data = np.load(path, allow_pickle=True)
    print(f'当前数据集所包含数据量：{data.shape[0]}, 每个数据都有 {data.shape[1]} 个数据，分别是图像数据和按键数据')
else:
    print('数据集不存在，以下代码无法运行')

In [ ]:
screen = data[0][0]    # 选取了第一个数据的图像部分
# print('图像', screen)
print('形状', screen.shape)

In [ ]:
action_value = data[0][1]    # 选取了第一个数据的按键记录部分
print('按键记录', action_value)
print('目前的训练只使用前五个，即独热编码部分  ', action_value[:5])

In [ ]:
# 统计动作分布
def motionCounts(Y):

    columns=['攻击', '弹反', '垫步', '跳跃', '无键', '道具', '向前', '向后', '向左', '向右']
    df = pd.DataFrame(Y, columns=columns, dtype=np.uint8)
    
    total = len(df)

    for motion in columns:
        motion_count = len(df)-df[motion].value_counts()[0]
        print(f'motion: {motion:>3}, 记录到的次数: {motion_count:>5}')

Y = np.array([i[1] for i in data])
motionCounts(Y)

In [ ]:
from pysekiro.get_vertices import roi

print()
print('以视频的形式展示数据。', '要你自己运行才看得见视频')
Remaining = len(data)
for screen, action_value in data:
    action = np.argmax(action_value[:5])

    if   action == 0:
        a = '攻击'
    elif action == 1:
        a = '弹反'
    elif action == 2:
        a = '垫步'
    elif action == 3:
        a = '跳跃'
    elif action == 4:
        a = '其他'

    cv2.imshow('screen', screen)
    cv2.imshow('roi', roi(screen, x=140, x_w=340, y=30, y_h=230))
    cv2.waitKey(1)

    Remaining -= 1
    print(f'\r 剩余: {Remaining:>4}, 动作:{a} {action_value}', end='') # end='\n'
    if cv2.waitKey(1) & 0xFF == ord('q'):    # 按 q 键关闭视频
        cv2.destroyAllWindows()
        break
else:
    cv2.destroyAllWindows()